# NQS on Ising model with NetKet
https://netket.readthedocs.io/en/latest/tutorials/gs-ising.html#neural-network-quantum-state
https://netket.readthedocs.io/en/latest/docs/varstate.html#expectation-values

In [7]:
import netket as nk
import flax.linen  as nn
from netket.operator.spin import sigmax, sigmaz

In [8]:
N = 20 # number of sites 
hi = nk.hilbert.Spin(s=1/2, N=N)

$\Gamma \sum_i \sigma^x_i$ transverse field term

In [9]:
gamma = 0 
H = sum([gamma * sigmax(hi, i) for i in range(N)])

$V \sum_i \sigma_i^z \sigma_{i+1}^z$ interaction term

In [10]:
V = - 1
H += sum([V * sigmaz(hi, i) * sigmaz(hi, (i + 1) % N) for i in range(N)])

Pick a model—in this case the restricted Boltzmann machine originally used in NQS—and a Monte Carlo sampler (strategy for proposing new configs), and initialize a variational quantum state.

In [11]:
rbm = nk.models.RBM(alpha=1, param_dtype=float, kernel_init=nn.initializers.normal(stddev=0.01))
sampler = nk.sampler.MetropolisLocal(hi)
vstate = nk.vqs.MCState(sampler, model=rbm, n_samples=500)


/Users/adamremels/miniconda3/envs/csci1470/lib/python3.10/site-packages/netket/vqs/mc/mc_state/state.py:257: UserWarning: n_samples=500 (500 per device/MPI rank) does not divide n_chains=16, increased to 512 (512 per device/MPI rank)
  self.n_samples = n_samples


Perform the actual variational optimization with stochastic gradient descent as an optimizer, and with stochastic reconfiguration as a preconditioner.

In [12]:
optimizer = nk.optimizer.Sgd(learning_rate=0.1)

# diag_shift is singular matrix regularization
gs = nk.driver.VMC(H, optimizer, variational_state=vstate, preconditioner=nk.optimizer.SR(diag_shift=0.1))

log=nk.logging.RuntimeLog()
gs.run(n_iter=300,out=log)

ffn_energy=vstate.expect(H)
print("Optimized energy:", ffn_energy)

100%|██████████| 300/300 [00:04<00:00, 71.79it/s, Energy=-2.000e+01 ± nan [σ²=0.000e+00]]         


Optimized energy: -2.000e+01 ± nan [σ²=0.000e+00]
